In [1]:
d1 = "The sky is blue."
d2 = "The sun is bright."
d3 = "The sun in the sky is bright."
d4 = "We can see the shining sun, the bright sun."
d5 = "Sun oh Mr. Sun, please shine on us. Your lovely sunshine makes my day bright"
d6 = "Sun, Sun go away, I hate sun, sun sun sun sun sun!"

original_documents = [d1, d2, d3, d4, d5, d6]

In [5]:
def tokenize(d):
    import re
    stopwords = open('stopwords.txt','r').read().splitlines()
    tokens = re.findall(r'\w+', d)
    return [t.lower() for t in tokens if t.lower() not in stopwords]

documents = []
for d in original_documents:
    documents.append(tokenize(d))
    
documents

[['sky', 'blue'],
 ['sun', 'bright'],
 ['sun', 'sky', 'bright'],
 ['shining', 'sun', 'bright', 'sun'],
 ['sun', 'sun', 'shine', 'lovely', 'sunshine', 'makes', 'day', 'bright'],
 ['sun', 'sun', 'hate', 'sun', 'sun', 'sun', 'sun', 'sun', 'sun']]

In [7]:
from collections import Counter
from math import log
doc_tf = []
for d in documents:
    length = len(d)
    tf = [(term, frequency / float(length)) for term, frequency in Counter(d).items()]
    doc_tf.append(tf)
# Calculate DF
N = len(documents) #Total documents
df = {}
for d in documents:
    for term in d:
        if term in df:
            continue
        else:
            df[term] = 0 #initialize the counter
        for doc in documents:
            if term in doc:
                df[term] = df[term] + 1.  # DON'T Forget the DOT
# Calculate IDF
idf = {}
for term in df:
    idf[term] = log(N/df[term])
features = []
for d in doc_tf:
    tfidf = [(term, frequency * idf[term]) for term,frequency in d]
    features.append(tfidf)
features

[('sky', 0.5), ('blue', 0.5)]
[('sun', 0.5), ('bright', 0.5)]
[('sun', 0.3333333333333333), ('sky', 0.3333333333333333), ('bright', 0.3333333333333333)]
[('shining', 0.25), ('sun', 0.5), ('bright', 0.25)]
[('sun', 0.25), ('shine', 0.125), ('lovely', 0.125), ('sunshine', 0.125), ('makes', 0.125), ('day', 0.125), ('bright', 0.125)]
[('sun', 0.8888888888888888), ('hate', 0.1111111111111111)]


[[('sky', 0.5493061443340549), ('blue', 0.8958797346140275)],
 [('sun', 0.0911607783969773), ('bright', 0.2027325540540822)],
 [('sun', 0.060773852264651526),
  ('sky', 0.3662040962227032),
  ('bright', 0.13515503603605478)],
 [('shining', 0.44793986730701374),
  ('sun', 0.0911607783969773),
  ('bright', 0.1013662770270411)],
 [('sun', 0.04558038919848865),
  ('shine', 0.22396993365350687),
  ('lovely', 0.22396993365350687),
  ('sunshine', 0.22396993365350687),
  ('makes', 0.22396993365350687),
  ('day', 0.22396993365350687),
  ('bright', 0.05068313851352055)],
 [('sun', 0.16206360603907075), ('hate', 0.19908438546978388)]]

In [8]:
import math
def cosim(vec1, vec2):
    v1 = dict(vec1)
    v2 = dict(vec2)
    intersection = set(v1.keys()) & set(v2.keys())
    numerator = sum([v1[x] * v1[x] for x in intersection])
    sum1 = sum([v1[x]**2 for x in v1.keys()])
    sum2 = sum([v2[x]**2 for x in v2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [9]:
import operator
query = 'sun bright'
q = [(token,1) for token in query.split(' ')]
results = {}
for i in range(len(features)):
    results[i] = cosim(features[i],q)
sorted_results = sorted(results.items(), key=operator.itemgetter(1), reverse = True)
for (i, score) in sorted_results:
    print("{0} - {1}".format(score,original_documents[i]))

0.15717947701788892 - The sun is bright.
0.07234623101347748 - Sun, Sun go away, I hate sun, sun sun sun sun sun!
0.039307037041059666 - The sun in the sky is bright.
0.028067366538722917 - We can see the shining sun, the bright sun.
0.006500346953948024 - Sun oh Mr. Sun, please shine on us. Your lovely sunshine makes my day bright
0.0 - The sky is blue.
